# Preparación del Escenario: El Reporte del Gerente

In [27]:
import pandas as pd
import numpy as np

# Reporte Ancho exportado del sistema antiguo de RRHH
data_rrhh = {
    'id_empleado': ['EMP-01', 'EMP-02', 'EMP-03', 'EMP-04', 'EMP-05'],
    'departamento': ['IT', 'IT', 'Ventas', 'Ventas', 'Marketing'],
    'rol': ['Backend', 'Frontend', 'Ejecutivo', 'Key Account', 'Diseñador'],
    'Ene_2024_Hrs': [160, 150, 120, 140, 100],
    'Feb_2024_Hrs': [165, 140, np.nan, 150, 110], # Alguien se ausentó
    'Mar_2024_Hrs': [155, 160, 130, 145, 115]
}

df_rrhh_ancho = pd.DataFrame(data_rrhh)
print("--- 🕒 REPORTE DE HORAS (FORMATO ANCHO) ---")
print(df_rrhh_ancho)

--- 🕒 REPORTE DE HORAS (FORMATO ANCHO) ---
  id_empleado departamento          rol  Ene_2024_Hrs  Feb_2024_Hrs  \
0      EMP-01           IT      Backend           160         165.0   
1      EMP-02           IT     Frontend           150         140.0   
2      EMP-03       Ventas    Ejecutivo           120           NaN   
3      EMP-04       Ventas  Key Account           140         150.0   
4      EMP-05    Marketing    Diseñador           100         110.0   

   Mar_2024_Hrs  
0           155  
1           160  
2           130  
3           145  
4           115  


# Solución 1: "Derretir" el Sistema Antiguo (melt)

In [28]:
# Se reestructura la tabla, 'id_empleado no cambia' por ende lo colocamos en 'id_vars', los meses se pasan a
df_largo=df_rrhh_ancho.melt(
    id_vars=['id_empleado','departamento','rol'],
    value_vars=['Ene_2024_Hrs', 'Feb_2024_Hrs', 'Mar_2024_Hrs'],
    var_name='mes_registro',
    value_name='horas_trabajadas'
)

print(df_largo.head(15))

   id_empleado departamento          rol  mes_registro  horas_trabajadas
0       EMP-01           IT      Backend  Ene_2024_Hrs             160.0
1       EMP-02           IT     Frontend  Ene_2024_Hrs             150.0
2       EMP-03       Ventas    Ejecutivo  Ene_2024_Hrs             120.0
3       EMP-04       Ventas  Key Account  Ene_2024_Hrs             140.0
4       EMP-05    Marketing    Diseñador  Ene_2024_Hrs             100.0
5       EMP-01           IT      Backend  Feb_2024_Hrs             165.0
6       EMP-02           IT     Frontend  Feb_2024_Hrs             140.0
7       EMP-03       Ventas    Ejecutivo  Feb_2024_Hrs               NaN
8       EMP-04       Ventas  Key Account  Feb_2024_Hrs             150.0
9       EMP-05    Marketing    Diseñador  Feb_2024_Hrs             110.0
10      EMP-01           IT      Backend  Mar_2024_Hrs             155.0
11      EMP-02           IT     Frontend  Mar_2024_Hrs             160.0
12      EMP-03       Ventas    Ejecutivo  Mar_2024_

# Solución 2: Limpiando la Basura en el Pipeline (Repaso .str)

In [29]:
df_nuevo = df_largo.assign(
    mes_registro=lambda x:x['mes_registro'].str.split("_").str[0],
    horas_trabajadas=lambda  x:x['horas_trabajadas'].fillna(0)
)
print(df_nuevo.head(15))

   id_empleado departamento          rol mes_registro  horas_trabajadas
0       EMP-01           IT      Backend          Ene             160.0
1       EMP-02           IT     Frontend          Ene             150.0
2       EMP-03       Ventas    Ejecutivo          Ene             120.0
3       EMP-04       Ventas  Key Account          Ene             140.0
4       EMP-05    Marketing    Diseñador          Ene             100.0
5       EMP-01           IT      Backend          Feb             165.0
6       EMP-02           IT     Frontend          Feb             140.0
7       EMP-03       Ventas    Ejecutivo          Feb               0.0
8       EMP-04       Ventas  Key Account          Feb             150.0
9       EMP-05    Marketing    Diseñador          Feb             110.0
10      EMP-01           IT      Backend          Mar             155.0
11      EMP-02           IT     Frontend          Mar             160.0
12      EMP-03       Ventas    Ejecutivo          Mar           

# Solución 3: La Tabla Dinámica del Director

In [30]:
df_resumen = df_nuevo.pivot_table(
    index = 'departamento', # Lo que irá en las filas (hacia abajo)
    columns = 'mes_registro', # Lo que irá en las columnas (hacia la derecha)
    values = 'horas_trabajadas', # Los números a calcular
    aggfunc = sum , # Sumamos las ventas
    fill_value = 0 # Si alguien no vendió nada, pon un 0 en vez de NaN
)

print(df_resumen)

mes_registro    Ene    Feb    Mar
departamento                     
IT            310.0  305.0  315.0
Marketing     100.0  110.0  115.0
Ventas        260.0  150.0  275.0


# Solución 4: El Reto Financiero Nivel Senior

In [34]:
df_senior = df_nuevo.pivot_table(
    index = 'rol', # Lo que irá en las filas (hacia abajo)
    columns = 'mes_registro', # Lo que irá en las columnas (hacia la derecha)
    values = 'horas_trabajadas', # Los números a calcular
    aggfunc = np.mean , # Sacamos el promedio
    fill_value = 0, # Si alguien no vendió nada, pon un 0 en vez de NaN
    margins=True,  # activa totales
    margins_name='Total General' # nombre de la fila/columna total
)

print("--- Totales Generales por rol y mes ----")
print(df_senior)

--- Totales Generales por rol y mes ----
mes_registro     Ene    Feb    Mar  Total General
rol                                              
Backend        160.0  165.0  155.0     160.000000
Diseñador      100.0  110.0  115.0     108.333333
Ejecutivo      120.0    0.0  130.0      83.333333
Frontend       150.0  140.0  160.0     150.000000
Key Account    140.0  150.0  145.0     145.000000
Total General  134.0  113.0  141.0     129.333333
